<a href="https://colab.research.google.com/github/manankapoor23/DSAI_Fine_Tuning/blob/main/01_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install beautifulsoup4 requests tqdm langdetect

import os
os.makedirs("data/raw", exist_ok=True)


In [5]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

def extract_article_text(url):
    try:
        r = requests.get(url, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")

        for tag in soup(["script", "style", "nav", "footer", "header"]):
            tag.decompose()

        text = soup.get_text(separator=" ")
        text = re.sub(r"\s+", " ", text)
        return text.strip()
    except:
        return None

def collect_news(site_url, limit=200):
    r = requests.get(site_url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    links = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("http"):
            links.add(href)

    articles = []
    for link in tqdm(list(links)[:limit], desc=f"Scraping {site_url}"):
        text = extract_article_text(link)
        if text and len(text) > 800:
            articles.append(text)

    return articles
news_sources = [
    "https://jagbani.punjabkesari.in/",
    "https://www.indiapress.org/gen/news.php/Daily_Ajit/400x60/0",
    "https://tv9punjabi.com/"
]

news_data = []

for site in news_sources:
    news_data.extend(collect_news(site, limit=250))

print("Collected news articles:", len(news_data))


Scraping https://jagbani.punjabkesari.in/: 100%|██████████| 242/242 [02:17<00:00,  1.76it/s]
Scraping https://www.indiapress.org/gen/news.php/Daily_Ajit/400x60/0: 0it [00:00, ?it/s]
Scraping https://tv9punjabi.com/: 100%|██████████| 145/145 [01:19<00:00,  1.83it/s]

Collected news articles: 304


In [6]:
with open("data/raw/punjabi_news.txt", "w", encoding="utf-8") as f:
    for article in news_data:
        f.write(article + "\n\n")

print("Punjabi news data saved successfully ✅")


Punjabi news data saved successfully ✅


In [7]:
import re
import unicodedata
import os

# Gurmukhi Unicode range
GURMUKHI_REGEX = re.compile(r'[\u0A00-\u0A7F]')

def clean_text(text):
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'[^A-Za-z\u0A00-\u0A7F\s।!?]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def is_punjabi_gurmukhi(text, threshold=0.7):
    if len(text) < 30:
        return False
    punjabi_chars = len(GURMUKHI_REGEX.findall(text))
    return (punjabi_chars / len(text)) >= threshold



In [8]:
input_file = "data/raw/punjabi_news.txt"

with open(input_file, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Loaded characters:", len(raw_text))


Loaded characters: 8714145


In [9]:
# Split aggressively (news pages are noisy)
chunks = re.split(r'[।!?]', raw_text)

clean_punjabi = []

for chunk in chunks:
    chunk = clean_text(chunk)
    if is_punjabi_gurmukhi(chunk):
        clean_punjabi.append(chunk)

print("Punjabi-only sentences kept:", len(clean_punjabi))


Punjabi-only sentences kept: 6551


In [10]:
for i in range(5):
    print(f"\nSample {i+1}:\n{clean_punjabi[i]}")



Sample 1:
Movie Trailers Home Movie Trailers Movie Trailers ਰਸ਼ਮਿਕਾ ਮੰਦਾਨਾ ਸਟਾਰਰ ਫਿਲਮ ਮਾਇਸਾ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਕਾਰਤਿਕ ਆਰੀਅਨ ਅਤੇ ਅਨੰਨਿਆ ਪਾਂਡੇ ਦੀ ਫਿਲਮ ਤੂੰ ਮੇਰੀ ਮੈਂ ਤੇਰਾ ਮੈਂ ਤੇਰਾ ਤੂੰ ਮੇਰੀ ਦਾ ਟ੍ਰੇਲਰ ਜਾਰੀ ਰਾਤ ਅਕੇਲੀ ਹੈ ਦਿ ਬਾਂਸਲ ਮਰਡਰਸ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਹਾਸੇ ਅਤੇ ਡਰ ਦਾ ਕਾਂਬੋ ਏਕਾਕੀ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਜ਼ਬਰਦਸਤ ਡਾਇਲਾਗ ਪ੍ਰੋਮੋ ਰਿਲੀਜ਼ ਮਸਤੀ ਦਾ ਰੰਗੀਨ ਪੋਸਟਰ ਰਿਲੀਜ਼ ਪਰੇਸ਼ ਰਾਵਲ ਦੀ ਫਿਲਮ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਰਸ਼ਮਿਕਾ ਮੰਦਾਨਾ ਦੀ ਫਿਲਮ ਦਿ ਗਰਲਫ੍ਰੈਂਡ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਪ੍ਰਭਾਸ ਦੀ ਫਿਲਮ ਦਿ ਰਾਜਾ ਸਾਬ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਸਤੰਬਰ ਨੂੰ ਰਿਲੀਜ਼ ਹੋਵੇਗਾ ਪ੍ਰਭਾਸ ਦੀ ਫਿਲਮ ਦਿ ਰਾਜਾ ਸਾਬ ਦਾ ਟ੍ਰੇਲਰ ਲਤਾ ਮੰਗੇਸ਼ਕਰ ਦੀ ਜੈਅੰਤੀ ਤੇ ਫਿਲਮ ਬਹਾਦੁਰ ਦਾ ਟੀਜ਼ਰ ਲਾਂਚ ਪਰੇਸ਼ ਰਾਵਲ ਦੀ ਫਿਲਮ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਟੀਜ਼ਰ ਪੋਸਟਰ ਰਿਲੀਜ਼ ਰਿਤਿਕ ਰੋਸ਼ਨ ਪ੍ਰਭਾਸ ਪ੍ਰਿਥਵੀਰਾਜ ਤੇ ਸ਼ਿਵਕਾਰਤੀਕੇਯਨ ਰਿਲੀਜ਼ ਕਰਨ ਫਿਲਮ ਕਾਂਤਾਰਾ ਚੈਪਟਰ ਦਾ ਟ੍ਰੇਲਰ ਇਸ ਦਿਨ ਰਿਲੀਜ਼ ਹੋਵੇਗਾ ਕੰਤਾਰਾ ਚੈਪਟਰ ਦਾ ਟ੍ਰੇਲਰ ਦਰਸ਼ਕਾਂ ਚ ਵਧਿਆ ਉਤਸ਼ਾਹ ਫਿਲਮ ਮੰਨੂ ਕਯਾ ਕਰੇਗਾ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ SS ਰਾਜਾਮੌਲੀ ਨੇ ਲਾਂਚ ਕੀਤਾ ਰਾਓ ਬਹਾਦੁਰ ਦਾ ਟੀਜ਼ਰ ਵਿਵੇਕ ਰੰਜਨ ਅਗਨੀਹੋਤਰੀ ਦੀ ਦਿ ਬੰਗਾਲ ਫਾਈਲਜ਼ ਦਾ ਟ੍ਰੇਲਰ ਹੋਇਆ ਰਿਲੀਜ਼ ਫਿਲਮ ਵਾਰ ਤੋਂ ਰਿਤਿਕ ਅਤੇ NTR ਜੂਨੀਅਰ

In [11]:
os.makedirs("data/processed", exist_ok=True)

output_file = "data/processed/punjabi_clean.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for sent in clean_punjabi:
        f.write(sent + "\n")

print("Saved clean Punjabi-only file ✅")


Saved clean Punjabi-only file ✅


In [12]:
total_chars = sum(len(s) for s in clean_punjabi)
approx_tokens = total_chars // 4   # LLaMA approx

print("Approx tokens:", approx_tokens)


Approx tokens: 266778


In [13]:
input_file = "data/processed/punjabi_clean.txt"

with open(input_file, "r", encoding="utf-8") as f:
    sentences = [line.strip() for line in f if line.strip()]

print("Sentences before deduplication:", len(sentences))


Sentences before deduplication: 6551


In [14]:
from hashlib import md5

def deduplicate(sentences):
    seen = set()
    unique = []

    for sent in sentences:
        h = md5(sent.encode("utf-8")).hexdigest()
        if h not in seen:
            seen.add(h)
            unique.append(sent)

    return unique

deduped_sentences = deduplicate(sentences)

print("Sentences after deduplication:", len(deduped_sentences))
print("Duplicates removed:", len(sentences) - len(deduped_sentences))


Sentences after deduplication: 4053
Duplicates removed: 2498


In [15]:
for i in range(5):
    print(f"\nSample {i+1}:\n{deduped_sentences[i]}")



Sample 1:
Movie Trailers Home Movie Trailers Movie Trailers ਰਸ਼ਮਿਕਾ ਮੰਦਾਨਾ ਸਟਾਰਰ ਫਿਲਮ ਮਾਇਸਾ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਕਾਰਤਿਕ ਆਰੀਅਨ ਅਤੇ ਅਨੰਨਿਆ ਪਾਂਡੇ ਦੀ ਫਿਲਮ ਤੂੰ ਮੇਰੀ ਮੈਂ ਤੇਰਾ ਮੈਂ ਤੇਰਾ ਤੂੰ ਮੇਰੀ ਦਾ ਟ੍ਰੇਲਰ ਜਾਰੀ ਰਾਤ ਅਕੇਲੀ ਹੈ ਦਿ ਬਾਂਸਲ ਮਰਡਰਸ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਹਾਸੇ ਅਤੇ ਡਰ ਦਾ ਕਾਂਬੋ ਏਕਾਕੀ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਜ਼ਬਰਦਸਤ ਡਾਇਲਾਗ ਪ੍ਰੋਮੋ ਰਿਲੀਜ਼ ਮਸਤੀ ਦਾ ਰੰਗੀਨ ਪੋਸਟਰ ਰਿਲੀਜ਼ ਪਰੇਸ਼ ਰਾਵਲ ਦੀ ਫਿਲਮ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਰਸ਼ਮਿਕਾ ਮੰਦਾਨਾ ਦੀ ਫਿਲਮ ਦਿ ਗਰਲਫ੍ਰੈਂਡ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਪ੍ਰਭਾਸ ਦੀ ਫਿਲਮ ਦਿ ਰਾਜਾ ਸਾਬ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਸਤੰਬਰ ਨੂੰ ਰਿਲੀਜ਼ ਹੋਵੇਗਾ ਪ੍ਰਭਾਸ ਦੀ ਫਿਲਮ ਦਿ ਰਾਜਾ ਸਾਬ ਦਾ ਟ੍ਰੇਲਰ ਲਤਾ ਮੰਗੇਸ਼ਕਰ ਦੀ ਜੈਅੰਤੀ ਤੇ ਫਿਲਮ ਬਹਾਦੁਰ ਦਾ ਟੀਜ਼ਰ ਲਾਂਚ ਪਰੇਸ਼ ਰਾਵਲ ਦੀ ਫਿਲਮ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਟੀਜ਼ਰ ਪੋਸਟਰ ਰਿਲੀਜ਼ ਰਿਤਿਕ ਰੋਸ਼ਨ ਪ੍ਰਭਾਸ ਪ੍ਰਿਥਵੀਰਾਜ ਤੇ ਸ਼ਿਵਕਾਰਤੀਕੇਯਨ ਰਿਲੀਜ਼ ਕਰਨ ਫਿਲਮ ਕਾਂਤਾਰਾ ਚੈਪਟਰ ਦਾ ਟ੍ਰੇਲਰ ਇਸ ਦਿਨ ਰਿਲੀਜ਼ ਹੋਵੇਗਾ ਕੰਤਾਰਾ ਚੈਪਟਰ ਦਾ ਟ੍ਰੇਲਰ ਦਰਸ਼ਕਾਂ ਚ ਵਧਿਆ ਉਤਸ਼ਾਹ ਫਿਲਮ ਮੰਨੂ ਕਯਾ ਕਰੇਗਾ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ SS ਰਾਜਾਮੌਲੀ ਨੇ ਲਾਂਚ ਕੀਤਾ ਰਾਓ ਬਹਾਦੁਰ ਦਾ ਟੀਜ਼ਰ ਵਿਵੇਕ ਰੰਜਨ ਅਗਨੀਹੋਤਰੀ ਦੀ ਦਿ ਬੰਗਾਲ ਫਾਈਲਜ਼ ਦਾ ਟ੍ਰੇਲਰ ਹੋਇਆ ਰਿਲੀਜ਼ ਫਿਲਮ ਵਾਰ ਤੋਂ ਰਿਤਿਕ ਅਤੇ NTR ਜੂਨੀਅਰ

In [18]:
deduped_file = "data/processed/punjabi_deduped.txt"

with open(deduped_file, "w", encoding="utf-8") as f:
    for sent in deduped_sentences:
        f.write(sent + "\n")

print("Saved deduplicated Punjabi text ✅")


Saved deduplicated Punjabi text ✅


In [19]:
import json
import os

os.makedirs("data/final", exist_ok=True)

jsonl_file = "data/final/punjabi_llama2.jsonl"

with open(jsonl_file, "w", encoding="utf-8") as f:
    for sent in deduped_sentences:
        record = {"text": sent}
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("Saved JSONL file for LLaMA-2 fine-tuning ✅")


Saved JSONL file for LLaMA-2 fine-tuning ✅


In [20]:
with open(jsonl_file, "r", encoding="utf-8") as f:
    for _ in range(3):
        print(f.readline())


{"text": "Movie Trailers Home Movie Trailers Movie Trailers ਰਸ਼ਮਿਕਾ ਮੰਦਾਨਾ ਸਟਾਰਰ ਫਿਲਮ ਮਾਇਸਾ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਕਾਰਤਿਕ ਆਰੀਅਨ ਅਤੇ ਅਨੰਨਿਆ ਪਾਂਡੇ ਦੀ ਫਿਲਮ ਤੂੰ ਮੇਰੀ ਮੈਂ ਤੇਰਾ ਮੈਂ ਤੇਰਾ ਤੂੰ ਮੇਰੀ ਦਾ ਟ੍ਰੇਲਰ ਜਾਰੀ ਰਾਤ ਅਕੇਲੀ ਹੈ ਦਿ ਬਾਂਸਲ ਮਰਡਰਸ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਹਾਸੇ ਅਤੇ ਡਰ ਦਾ ਕਾਂਬੋ ਏਕਾਕੀ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਜ਼ਬਰਦਸਤ ਡਾਇਲਾਗ ਪ੍ਰੋਮੋ ਰਿਲੀਜ਼ ਮਸਤੀ ਦਾ ਰੰਗੀਨ ਪੋਸਟਰ ਰਿਲੀਜ਼ ਪਰੇਸ਼ ਰਾਵਲ ਦੀ ਫਿਲਮ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਰਸ਼ਮਿਕਾ ਮੰਦਾਨਾ ਦੀ ਫਿਲਮ ਦਿ ਗਰਲਫ੍ਰੈਂਡ ਦਾ ਟੀਜ਼ਰ ਰਿਲੀਜ਼ ਪ੍ਰਭਾਸ ਦੀ ਫਿਲਮ ਦਿ ਰਾਜਾ ਸਾਬ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ ਸਤੰਬਰ ਨੂੰ ਰਿਲੀਜ਼ ਹੋਵੇਗਾ ਪ੍ਰਭਾਸ ਦੀ ਫਿਲਮ ਦਿ ਰਾਜਾ ਸਾਬ ਦਾ ਟ੍ਰੇਲਰ ਲਤਾ ਮੰਗੇਸ਼ਕਰ ਦੀ ਜੈਅੰਤੀ ਤੇ ਫਿਲਮ ਬਹਾਦੁਰ ਦਾ ਟੀਜ਼ਰ ਲਾਂਚ ਪਰੇਸ਼ ਰਾਵਲ ਦੀ ਫਿਲਮ ਦਿ ਤਾਜ ਸਟੋਰੀ ਦਾ ਟੀਜ਼ਰ ਪੋਸਟਰ ਰਿਲੀਜ਼ ਰਿਤਿਕ ਰੋਸ਼ਨ ਪ੍ਰਭਾਸ ਪ੍ਰਿਥਵੀਰਾਜ ਤੇ ਸ਼ਿਵਕਾਰਤੀਕੇਯਨ ਰਿਲੀਜ਼ ਕਰਨ ਫਿਲਮ ਕਾਂਤਾਰਾ ਚੈਪਟਰ ਦਾ ਟ੍ਰੇਲਰ ਇਸ ਦਿਨ ਰਿਲੀਜ਼ ਹੋਵੇਗਾ ਕੰਤਾਰਾ ਚੈਪਟਰ ਦਾ ਟ੍ਰੇਲਰ ਦਰਸ਼ਕਾਂ ਚ ਵਧਿਆ ਉਤਸ਼ਾਹ ਫਿਲਮ ਮੰਨੂ ਕਯਾ ਕਰੇਗਾ ਦਾ ਟ੍ਰੇਲਰ ਰਿਲੀਜ਼ SS ਰਾਜਾਮੌਲੀ ਨੇ ਲਾਂਚ ਕੀਤਾ ਰਾਓ ਬਹਾਦੁਰ ਦਾ ਟੀਜ਼ਰ ਵਿਵੇਕ ਰੰਜਨ ਅਗਨੀਹੋਤਰੀ ਦੀ ਦਿ ਬੰਗਾਲ ਫਾਈਲਜ਼ ਦਾ ਟ੍ਰੇਲਰ ਹੋਇਆ ਰਿਲੀਜ਼ ਫਿਲਮ ਵਾਰ ਤੋਂ ਰਿਤਿਕ ਅਤੇ NTR ਜੂਨੀਅਰ 

In [21]:
total_chars = sum(len(s) for s in deduped_sentences)
approx_tokens = total_chars // 4

print("Approx tokens:", approx_tokens)


Approx tokens: 144108


In [22]:
print("Average sentence length:",
      sum(len(s) for s in deduped_sentences) / len(deduped_sentences))


Average sentence length: 142.22378485072787


In [24]:
pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=1d4c6b25667dbd1d71a146a5fbf0f303968e349392786241a4bf7ece9be2253a
  Stored in directory: /root/.cache/pip/wheels/92/a8/b7/d8a067c31a74de9ca252bbe53dea5f896faabd25d55f541037
Successfully built GPUtil


In [1]:


with open("/content/drive/MyDrive/punjabi_llm_project/data/final/train_safe_v1.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        try:
            json.loads(line)
        except Exception as e:
            print("Error on line", i, e)


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/punjabi_llm_project/data/final/train_safe_v1.jsonl'

In [2]:
pip install transformers sentencepiece


In [ ]:
from huggingface_hub import notebook_login
notebook_login()



In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    use_fast=False
)


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [7]:
import json

file_path = "/Users/manankapoor/finetune_llm/DSAI_Fine_Tuning/final/train_safe_v1.jsonl"

total_tokens = 0
num_samples = 0
max_len = 0

with open(file_path, "r", encoding="utf-8") as f:
    for line in f:
        obj = json.loads(line)
        text = obj["text"]

        tokens = tokenizer.encode(text)
        token_count = len(tokens)

        total_tokens += token_count
        num_samples += 1
        max_len = max(max_len, token_count)

print("Samples:", num_samples)
print("Total tokens:", total_tokens)
print("Average tokens per sample:", total_tokens // num_samples)
print("Max tokens in a sample:", max_len)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/manankapoor/finetune_llm/DSAI_Fine_Tuning/final/train_safe_v1.jsonl'